In [8]:
import pandas as pd
import csv

In [9]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# get the first table from the website
response = pd.read_html(url)[0] 

# write the data from website to a csv file
response.to_csv(r'rawdata.csv', mode='a',encoding='UTF-8',header=1,index=0) 


In [10]:
# read the data in csv file to a dataframe
df = pd.read_csv(r'rawdata.csv')

In [11]:
# get rid of the "Not assigned" value of Column BOROUGH
df = df[df['Borough'] != "Not assigned"]


In [12]:
df.reset_index(drop=True, inplace=True)
# update the value of Column "Neighbourhood"
df.replace(['Neighbourhood','Not assigned'],'Borough')
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
#  group the Neighborhood by same postcode and Borough
df = df.groupby(['Postcode','Borough'],sort=False).agg(','.join).reset_index()
df.head()
df.shape

(104, 3)

In [14]:
# download the csv file and store as dataframe
csv = 'http://cocl.us/Geospatial_data'
df1 = pd.read_csv(csv)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df1 = df1.rename(columns={'Postal Code':'Postcode'})
df1.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# merge 2 tables
geo_data = df.merge(df1,on=['Postcode'])
geo_data.shape

(103, 5)

In [17]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [18]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim


Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.03=py37_0
  - defaults/win-64::numba==0.43.1=py37hf9181ef_0
done

## Package Plan ##

  environment location: D:\Anaconda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           5 KB
    anaconda-custom            |           py37_1           2 KB
    ca-certificates-2019.9.11  |       hecc5488_0         181 KB  conda-forge
    certifi-2019.6.16          |           py37_1         149 KB  conda-forge
    openssl-1.1.1c             |       hfa6e2cd_0         4.7 MB  conda-forge
    tbb-2019.9                 |       he980bc4_0         161 KB  conda-forge
    ------------------------------------------------

In [19]:
address = "Toronto"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinator of Toronto are {}, {}.".format(latitude,longitude))

The geographical coordinator of Toronto are 43.653963, -79.387207.


In [22]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

# add markers to map
for lat,lng,borough,neighbourhood in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Borough'], geo_data['Neighbourhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto